In [1]:
# import library for Support Vector Classification
from sklearn.svm import SVC

# import library for plot
import matplotlib.pyplot as plt

# import library for data handling
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import math

# import library for pytorch
import torch
import torch.nn as nn

# import custom function
from pytorch_test.forcast.stockLSTM import SequenceDataset, ShallowRegressionLSTM, train, test, get_stock_data, split_train_test

# Get cuda device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# Get the past stock price
df = get_stock_data(stock_name = 'TSLA', source = 'yahoo', past_year = 2)

# Split data into train and test
train_df, test_df, feature, target, train_size, test_size, target_mean, target_std = split_train_test(training_target = "Close", df = df, train_ratio = 0.8)


In [3]:
# create dataset and data loader
batch_size = 4
seq_len = 60

torch.manual_seed(seq_len*batch_size/2)

train_dataset = SequenceDataset(train_df, target, feature, seq_len)
test_dataset = SequenceDataset(test_df, target, feature, seq_len)
train_looader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# get the first batch and its shape
x, y = next(iter(train_looader))
print("Features shape: ", x.shape)
print("Target shape: ", y.shape)

Features shape:  torch.Size([4, 60, 5])
Target shape:  torch.Size([4])


In [4]:
# create model, loss and optimizer
learning_rate = 5e-5
num_hidden = 16
epochs = 40

model = ShallowRegressionLSTM(n_features = len(feature), n_hidden = num_hidden).to(device)
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
# train and test model
print("Untrained test\n----------------")
test(model, test_loader, loss, device)
print("\nTraining\n----------------")

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n")
    train(model, train_looader, loss, optimizer, device)
    test(model, test_loader, loss, device)

Untrained test
----------------
Test loss: 0.4078

Training
----------------
Epoch 1

Train loss: 1.0073
Test loss: 0.3780
Epoch 2

Train loss: 0.9547
Test loss: 0.3494
Epoch 3

Train loss: 0.8985
Test loss: 0.3201
Epoch 4

Train loss: 0.8472
Test loss: 0.2910
Epoch 5

Train loss: 0.7909
Test loss: 0.2592
Epoch 6

Train loss: 0.7322
Test loss: 0.2251
Epoch 7

Train loss: 0.6705
Test loss: 0.1904
Epoch 8

Train loss: 0.6025
Test loss: 0.1552
Epoch 9

Train loss: 0.5378
Test loss: 0.1222
Epoch 10

Train loss: 0.4732
Test loss: 0.0937
Epoch 11

Train loss: 0.4131
Test loss: 0.0738
Epoch 12

Train loss: 0.3660
Test loss: 0.0619
Epoch 13

Train loss: 0.3314
Test loss: 0.0569
Epoch 14

Train loss: 0.2951
Test loss: 0.0569
Epoch 15

Train loss: 0.2679
Test loss: 0.0597
Epoch 16

Train loss: 0.2456
Test loss: 0.0639
Epoch 17

Train loss: 0.2268
Test loss: 0.0683
Epoch 18

Train loss: 0.2088
Test loss: 0.0737
Epoch 19

Train loss: 0.1948
Test loss: 0.0781
Epoch 20

Train loss: 0.1813
Test loss: